In [2]:
pip install numpy scipy pythran bitarray

     |████████████████████████████████| 4.3 MB 7.6 MB/s 
     |████████████████████████████████| 88 kB 7.2 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
  Created wheel for bitarray: filename=bitarray-2.3.4-cp37-cp37m-linux_x86_64.whl size=171960 sha256=f4457166ba303680dca16491b94d8089d0041c5fca950fd470f0a1a279c0d83f
  Stored in directory: /root/.cache/pip/wheels/84/cc/5b/0e861bdb5294d22d2d4f595df936f964a95258387e11494d41
Successfully built bitarray
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires gast==0.4.0, but you have gast 0.5.2 which is incompatible.


In [3]:
!git clone https://github.com/ChalmersPhotonicsLab/QAMpy.git

Cloning into 'QAMpy'...
remote: Enumerating objects: 9866, done.
remote: Counting objects: 100% (1605/1605), done.
remote: Compressing objects: 100% (519/519), done.
remote: Total 9866 (delta 1190), reused 1479 (delta 1083), pack-reused 8261
Receiving objects: 100% (9866/9866), 225.76 MiB | 29.78 MiB/s, done.
Resolving deltas: 100% (6781/6781), done.


In [4]:
cd QAMpy/

/content/QAMpy


In [5]:
!python setup.py build

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/qampy
copying qampy/helpers.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/filtering.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/phaserec.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/analog_frontend.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/theory.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/equalisation.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/__init__.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/impairments.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/io.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/signals.py -> build/lib.linux-x86_64-3.7/qampy
creating build/lib.linux-x86_64-3.7/qampy/core
copying qampy/core/pilotbased_receiver.py -> build/lib.linux-x86_64-3.7/qampy/core
copying qampy/core/phaserecovery.py -> build/lib.linux-x86_64-3.7/qampy/core
copying qampy/core/ber_functions.py -> build

In [6]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating qampy.egg-info
writing qampy.egg-info/PKG-INFO
writing dependency_links to qampy.egg-info/dependency_links.txt
writing requirements to qampy.egg-info/requires.txt
writing top-level names to qampy.egg-info/top_level.txt
writing manifest file 'qampy.egg-info/SOURCES.txt'
adding license file 'COPYING'
writing manifest file 'qampy.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/helpers.py -> build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/filtering.py -> build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/phaserec.py -> build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/analog_frontend.py -> build/bdist.linux-x86_64/e

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
def KramersKronigRx(Amp, Nsamples, Fs):
    """
    :param Amp: signal amplitude value
    :param Nsamples: size of the FFT
    :param Fs: signal's sampling frequency
    :return phiTime: calculated signal's phase values
    """
    T  = 1/Fs 
    xf = fftfreq(Nsamples, Fs)
    xf = fftshift(xf)                
                       
    phiOmega = 1j*np.sign(xf)*fft(np.log(Amp))
    
    phiTime  = ifft(phiOmega)
    
    return np.real(phiTime)

In [9]:
from Funcoes import *
from numpy.fft import fft,fftfreq, fftshift, ifft

In [10]:
M = 16        # ordem da modulação
Fb = 24e9      # taxa de símbolos
SpS = 2         # amostras por símbolo
Fs = SpS*Fb    # taxa de amostragem
rolloff = 0.05  # Rolloff do filtro formatador de pulso
amostras = 512
size = 110000

In [15]:
SNRs = np.arange(18,49,2)
BERsKK = np.zeros(SNRs.size)
KK_64QAM = {}
fft_size = size
A = 10
M = 64
for SpS in [2,4,8]:
  BERsKK = np.zeros(SNRs.size)
  for i , SNR in enumerate(SNRs): 
    qam_signal = generate_signal(M,Fb,SpS,SNR,rolloff=0.05)
    sfm, _,_ = qam_signal_phase_min(qam_signal,A)
    data,X,y = dataset_02(sfm,amostras,size)
    phases = KramersKronigRx(data['amplitudes'],fft_size,Fs)
    predicted_kk_signal = data['amplitudes']*np.exp(1j*phases.reshape(-1,))
    reverted_signal = revert_sfm(predicted_kk_signal.reshape((1,-1)),A)
    BERsKK[i] = reverted_signal[0,5000:(size-5000):SpS].cal_ber()
  KK_64QAM[f'BER_KK_{SpS}_sps'] = BERsKK
  print(KK_64QAM)
  print('\n\n')

{'BER_KK_2_sps': array([0.08734333, 0.06464   , 0.04883333, 0.03425333, 0.02739667,
       0.01992   , 0.02018333, 0.01742   , 0.01791667, 0.01633333,
       0.01577667, 0.01565   , 0.01644   , 0.01436   , 0.01578333,
       0.01547333])}



{'BER_KK_2_sps': array([0.08734333, 0.06464   , 0.04883333, 0.03425333, 0.02739667,
       0.01992   , 0.02018333, 0.01742   , 0.01791667, 0.01633333,
       0.01577667, 0.01565   , 0.01644   , 0.01436   , 0.01578333,
       0.01547333]), 'BER_KK_4_sps': array([0.07636   , 0.05067333, 0.03043333, 0.01818   , 0.00970667,
       0.00742   , 0.00463333, 0.00457333, 0.00237333, 0.00398667,
       0.00274   , 0.00298   , 0.00249333, 0.00192   , 0.00238667,
       0.00282   ])}



{'BER_KK_2_sps': array([0.08734333, 0.06464   , 0.04883333, 0.03425333, 0.02739667,
       0.01992   , 0.02018333, 0.01742   , 0.01791667, 0.01633333,
       0.01577667, 0.01565   , 0.01644   , 0.01436   , 0.01578333,
       0.01547333]), 'BER_KK_4_sps': array([0.07636   , 0.05

In [16]:
KK_64QAM

{'BER_KK_2_sps': array([0.08734333, 0.06464   , 0.04883333, 0.03425333, 0.02739667,
        0.01992   , 0.02018333, 0.01742   , 0.01791667, 0.01633333,
        0.01577667, 0.01565   , 0.01644   , 0.01436   , 0.01578333,
        0.01547333]),
 'BER_KK_4_sps': array([0.07636   , 0.05067333, 0.03043333, 0.01818   , 0.00970667,
        0.00742   , 0.00463333, 0.00457333, 0.00237333, 0.00398667,
        0.00274   , 0.00298   , 0.00249333, 0.00192   , 0.00238667,
        0.00282   ]),
 'BER_KK_8_sps': array([0.07398667, 0.05132   , 0.03134667, 0.01832   , 0.0104    ,
        0.00529333, 0.00366667, 0.00274667, 0.00546667, 0.00462667,
        0.0032    , 0.00164   , 0.00361333, 0.00397333, 0.0034    ,
        0.00498667])}

In [ ]:
BERsKK

array([0.22460333, 0.18311   , 0.14638667, 0.11391   , 0.08741667,
       0.06467333, 0.04809333, 0.03718333, 0.02931   , 0.02167333,
       0.02030667])

In [ ]:
KK_data = {}

In [ ]:
KK_data['BER_KK_8_sps'] = BERsKK

In [ ]:
KK_data

{'BER_KK_2_sps': array([0.22460333, 0.18311   , 0.14638667, 0.11391   , 0.08741667,
        0.06467333, 0.04809333, 0.03718333, 0.02931   , 0.02167333,
        0.02030667]),
 'BER_KK_4_sps': array([0.22251333, 0.17965333, 0.14215333, 0.10771333, 0.07649333,
        0.05073333, 0.03168667, 0.01877333, 0.01036667, 0.00695333,
        0.00594667]),
 'BER_KK_8_sps': array([0.22098667, 0.17708   , 0.13861333, 0.10622667, 0.07406667,
        0.0496    , 0.03136   , 0.02108   , 0.01204   , 0.00801333,
        0.00564   ])}

In [18]:
import pickle

In [19]:
a_file = open("BER_KK_17_CSPR_64QAM.pkl", "wb")
pickle.dump(KK_64QAM, a_file)
a_file.close()

In [ ]:
KK_cspr = {}

In [ ]:
KK_cspr = {}
M = 64
SNRs = np.arange(18,49,2)
BERsKK = np.zeros(SNRs.size)
SpS = 8
fft_size = size
for CSPR_log in range(4,17):
  CSPR = 10**(CSPR_log/10)
  print('>'*10+' ' + str(CSPR) + ' ' + '<'*10)
  BERsKK = np.zeros(SNRs.size)
  for i , SNR in enumerate(SNRs):
      qam_signal = generate_signal(M,Fb,SpS,SNR,rolloff=0.05)
      A = np.sqrt(2*CSPR*np.mean(np.abs(qam_signal**2)))
      sfm, _,_ = qam_signal_phase_min(qam_signal,A)
      data,X,y = dataset_02(sfm,amostras,size)
      phases = KramersKronigRx(data['amplitudes'],fft_size,Fs)
      predicted_kk_signal = data['amplitudes']*np.exp(1j*phases.reshape(-1,))
      reverted_signal = revert_sfm(predicted_kk_signal.reshape((1,-1)),A)
      BERsKK[i] = reverted_signal[0,5000:(size-5000):SpS].cal_ber()
  #print(BERsKK)
  KK_cspr[f'KK_{SpS}_sps_{CSPR_log}_CSPR'] = BERsKK
  print(KK_cspr) 

>>>>>>>>>> 2.51188643150958 <<<<<<<<<<
{'KK_8_sps_4_CSPR': array([0.08585333, 0.06116   , 0.03885333, 0.02688   , 0.01497333,
       0.0118    , 0.00994667, 0.00933333, 0.01317333, 0.00602667,
       0.00750667, 0.00796   , 0.00777333, 0.00582667, 0.00812   ,
       0.00762667])}
>>>>>>>>>> 3.1622776601683795 <<<<<<<<<<
{'KK_8_sps_4_CSPR': array([0.08585333, 0.06116   , 0.03885333, 0.02688   , 0.01497333,
       0.0118    , 0.00994667, 0.00933333, 0.01317333, 0.00602667,
       0.00750667, 0.00796   , 0.00777333, 0.00582667, 0.00812   ,
       0.00762667]), 'KK_8_sps_5_CSPR': array([0.08234667, 0.05637333, 0.03701333, 0.02308   , 0.01372   ,
       0.01077333, 0.00841333, 0.00696   , 0.00750667, 0.00710667,
       0.00588   , 0.00761333, 0.00737333, 0.00712   , 0.00645333,
       0.00804   ])}
>>>>>>>>>> 3.9810717055349722 <<<<<<<<<<
{'KK_8_sps_4_CSPR': array([0.08585333, 0.06116   , 0.03885333, 0.02688   , 0.01497333,
       0.0118    , 0.00994667, 0.00933333, 0.01317333, 0.00602667,


In [ ]:
KK_cspr

{'KK_8_sps_10_CSPR': array([0.07752   , 0.05190667, 0.03357333, 0.01993333, 0.01036   ,
        0.00604   , 0.00457333, 0.00558667, 0.00497333, 0.00310667,
        0.00377333, 0.00444   , 0.00337333, 0.00392   , 0.00428   ,
        0.00352   ]),
 'KK_8_sps_11_CSPR': array([0.07733333, 0.05181333, 0.03153333, 0.01825333, 0.01304   ,
        0.00974667, 0.00684   , 0.00634667, 0.00397333, 0.00349333,
        0.00449333, 0.00456   , 0.00230667, 0.00674667, 0.00404   ,
        0.00329333]),
 'KK_8_sps_12_CSPR': array([0.0766    , 0.05229333, 0.03341333, 0.01886667, 0.01305333,
        0.00746667, 0.00561333, 0.00572   , 0.00457333, 0.00322667,
        0.00433333, 0.0026    , 0.00261333, 0.00306667, 0.00545333,
        0.00253333]),
 'KK_8_sps_13_CSPR': array([0.07557333, 0.05089333, 0.03130667, 0.0184    , 0.01233333,
        0.00749333, 0.0078    , 0.00376   , 0.00377333, 0.00426667,
        0.00497333, 0.00365333, 0.00329333, 0.00390667, 0.00253333,
        0.00345333]),
 'KK_8_sps_14_CS

In [ ]:
import pickle
a_file = open(f"BER_KK_different_CSPRs_{SpS}_SpS_{M}QAM.pkl", "wb")
pickle.dump(KK_cspr, a_file)
a_file.close()

In [ ]:
CSPR_log = np.arange(4,17)
CSPR = 10**(CSPR_log/10)
np.sqrt(2*CSPR*np.mean(np.abs(qam_signal**2)))

ResampledQAM([2.24979564, 2.52431222, 2.8323249 , 3.17792081, 3.56568579,
              4.00076526, 4.48893245, 5.03666505, 5.65123114, 6.34078563,
              7.11447849, 7.98257615, 8.95659776])

In [ ]:
np.mean(np.abs(qam_signal**2))

ResampledQAM(1.00752573)

In [ ]:
10*np.log10(10**2/2)

16.989700043360187

In [ ]:
10**2/2

50.0